In [1]:
%pip install -q kor markdownify requests pydantic pydantic[email] openai colorama bs4 rich

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 3.0 MB/s eta 0:00:00


In [2]:
from pydantic import BaseModel, Field, ValidationError

from math import pi


class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int = Field(default=1, gt=0)

    def area(self):
        return pi * self.radius ** 2

c = Circle(center=(1, 1), radius=2)

In [3]:
c.model_dump()

{'center': (1, 1), 'radius': 2}

So, while we can add properties to our pydantic models, they do not get serialized. We'll need to do a bit more work if we want those properties to be included in our model serialization.

Before we look at that, I want to point out that we could also choose to cache our properties.

In this example, we might want to cache the area - but that means we need to make sure the radius field is immutable (frozen). We could make the entire circle frozen, or we could choose to just make radius frozen.



In [4]:
from functools import cached_property

class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int = Field(default=1, gt=0, frozen=True)

    @cached_property
    def area(self):
        print("calculating area...")
        return pi * self.radius ** 2

c = Circle()
c.area

calculating area...


3.141592653589793

## Computed Fields
Sometimes, we want those properties to be actual fields in our Pydantic model - which means they get serialized just like any other field.

In [5]:
from functools import cached_property
from math import pi
from pydantic import BaseModel, computed_field, Field, PydanticUserError, ValidationError

In [6]:
try:
    class Circle(BaseModel):
        center: tuple[int, int] = (0, 0)
        radius: int = Field(default=1, gt=0, frozen=True)

        @computed_field
        @property
        def area(self):
            print("calculating area...")
            return pi * self.radius ** 2
except PydanticUserError as ex:
    print(ex)

Computed field is missing return type annotation or specifying `return_type` to the `@computed_field` decorator (e.g. `@computed_field(return_type=int|str)`)

For further information visit https://errors.pydantic.dev/2.6/u/model-field-missing-annotation


Now, with these computed properties, we do not have that Field mechanism to omit a computed field from the representation. But the @computed_field decorator, does enable that option, also using repr as an argument:

In [7]:
class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int = Field(default=1, gt=0, frozen=True)

    @computed_field(alias="AREA", repr=False)
    @property
    def area(self) -> float:
        print("calculating area...")
        return pi * self.radius ** 2

In [8]:
c = Circle()

In [10]:
c.area

calculating area...


3.141592653589793

In [12]:
c.model_dump()

calculating area...


{'center': (0, 0), 'radius': 1, 'area': 3.141592653589793}

In [13]:
try:
    c.area = 10
except AttributeError as ex:
    print(ex)

can't set attribute 'area'


In [14]:
class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int = Field(default=1, gt=0, frozen=True)

    @computed_field(alias="AREA", repr=False)
    @cached_property
    def area(self) -> float:
        print("calculating area...")
        return pi * self.radius ** 2

c = Circle()
c.area

calculating area...


3.141592653589793

## PROJECT

In [15]:
countries = {
    "australia": ("Australia", "AUS"),
    "canada": ("Canada", "CAN"),
    "china": ("China", "CHN"),
    "france": ("France", "FRA"),
    "germany": ("Germany", "DEU"),
    "india": ("India", "IND"),
    "mexico": ("Mexico", "MEX"),
    "norway": ("Norway", "NOR"),
    "pakistan": ("Pakistan", "PAK"),
    "san marino": ("San Marino", "SMR"),
    "sanmarino": ("San Marino", "SMR"),
    "spain": ("Spain", "ESP"),
    "sweden": ("Sweden", "SWE"),
    "united kingdom": ("United Kingdom", "GBR"),
    "uk": ("United Kingdom", "GBR"),
    "great britain": ("United Kingdom", "GBR"),
    "britain": ("United Kingdom", "GBR"),
    "us": ("United States of America", "USA"),
    "united states": ("United States of America", "USA"),
    "usa": ("United States of America", "USA"),
}
valid_country_names = sorted(countries.keys())
valid_country_names

def lookup_country(name: str) -> tuple[str, str]:
    name = name.strip().casefold()

    try:
        return countries[name]
    except KeyError:
        raise ValueError(
            "Unknown country name. "
            f"Country name must be one of: {','.join(valid_country_names)}"
        )

In [16]:
from datetime import date
from enum import Enum
from typing import Annotated, TypeVar
from uuid import uuid4
from pydantic import (
    AfterValidator,
    BaseModel,
    ConfigDict,
    Field,
    UUID4,
    field_serializer,
    field_validator,
    ValidationInfo,
)
from pydantic.alias_generators import to_camel


class AutomobileType(Enum):
    sedan = "Sedan"
    coupe = "Coupe"
    convertible = "Convertible"
    suv = "SUV"
    truck = "Truck"


T = TypeVar('T')
BoundedString = Annotated[str, Field(min_length=2, max_length=50)]
BoundedList = Annotated[list[T], Field(min_length=1, max_length=5)]


Country = Annotated[str, AfterValidator(lambda name: lookup_country(name)[0])]

class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel,
    )

    id_: UUID4 | None = Field(alias="id", default_factory=uuid4)
    manufacturer: BoundedString
    series_name: BoundedString
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = False
    manufactured_date: date = Field(validation_alias="completionDate", ge=date(1980, 1, 1))
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD",
        serialization_alias="baseMSRPUSD"
    )
    top_features: BoundedList[BoundedString] | None = None
    vin: BoundedString
    number_of_doors: int = Field(
        default=4,
        validation_alias="doors",
        ge=2,
        le=4,
        multiple_of=2,
    )
    registration_country: Country | None = None
    registration_date: date | None = None
    license_plate: BoundedString | None = None

    @field_serializer("manufactured_date", "registration_date", when_used="json-unless-none")
    def serialize_date(self, value: date) -> str:
        return value.strftime("%Y/%m/%d")

    @field_validator("registration_date")
    @classmethod
    def validate_registration_date(cls, value:date, values: ValidationInfo):
        data = values.data
        if "manufactured_date" in data and data["manufactured_date"] > value:
            raise ValueError("Automobile cannot be registered prior to manufacture date.")
        return value

In [17]:
country_code_lookup = {
    name: code
    for name, code in countries.values()
}
country_code_lookup

{'Australia': 'AUS',
 'Canada': 'CAN',
 'China': 'CHN',
 'France': 'FRA',
 'Germany': 'DEU',
 'India': 'IND',
 'Mexico': 'MEX',
 'Norway': 'NOR',
 'Pakistan': 'PAK',
 'San Marino': 'SMR',
 'Spain': 'ESP',
 'Sweden': 'SWE',
 'United Kingdom': 'GBR',
 'United States of America': 'USA'}

Also, now would be a good time to clean up our model's representation string. Remove all fields, except for id_, manufacturer, series_name and type_ from the representatino string.

Here is some sample data to help you test your model:

In [18]:
from uuid import UUID

data = {
    "id": "c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7",
    "manufacturer": "BMW",
    "seriesName": "M4 Competition xDrive",
    "type": "Convertible",
    "isElectric": False,
    "completionDate": "2023-01-01",
    "msrpUSD": 93_300,
    "topFeatures": ["6 cylinders", "all-wheel drive", "convertible"],
    "vin": "1234567890",
    "doors": 2,
    "registrationCountry": "us",
    "registrationDate": "2023-06-01",
    "licensePlate": "AAA-BBB"
}

expected_serialized_by_alias = {
    'id': UUID('c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7'),
    'manufacturer': 'BMW',
    'seriesName': 'M4 Competition xDrive',
    'type': AutomobileType.convertible,
    'isElectric': False,
    'manufacturedDate': date(2023, 1, 1),
    'baseMSRPUSD': 93300.0,
    'topFeatures': ['6 cylinders', 'all-wheel drive', 'convertible'],
    'vin': '1234567890',
    'numberOfDoors': 2,
    'registrationCountry': 'United States of America',
    'registrationCountryCode': 'USA',
    'registrationDate': date(2023, 6, 1),
    'licensePlate': 'AAA-BBB',
}

## Solution
To implement this we are going to use ```@cached_property``` and Pydantic's ```@computed_field decorators```.

In [19]:
from functools import cached_property
from pydantic import computed_field

class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel,
    )

    id_: UUID4 | None = Field(alias="id", default_factory=uuid4)
    manufacturer: BoundedString
    series_name: BoundedString
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = Field(default=False, repr=False)
    manufactured_date: date = Field(
        validation_alias="completionDate",
        ge=date(1980, 1, 1),
        repr=False
    )
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD",
        serialization_alias="baseMSRPUSD",
        repr=False,
    )
    top_features: BoundedList[BoundedString] | None = Field(default=None, repr=False)
    vin: BoundedString = Field(repr=False)
    number_of_doors: int = Field(
        default=4,
        validation_alias="doors",
        ge=2,
        le=4,
        multiple_of=2,
        repr=False,
    )
    registration_country: Country | None = Field(default=None, repr=False)

    @computed_field(repr=False)
    @cached_property
    def registration_country_code(self) -> str:
        return country_code_lookup[self.registration_country]

    registration_date: date | None = Field(default=None, repr=False)
    license_plate: BoundedString | None = Field(default=None, repr=False)


    @field_serializer("manufactured_date", "registration_date", when_used="json-unless-none")
    def serialize_date(self, value: date) -> str:
        return value.strftime("%Y/%m/%d")

    @field_validator("registration_date")
    @classmethod
    def validate_registration_date(cls, value:date, values: ValidationInfo):
        data = values.data
        if "manufactured_date" in data and data["manufactured_date"] > value:
            raise ValueError("Automobile cannot be registered prior to manufacture date.")
        return value

In [20]:
car = Automobile.model_validate(data)
car

Automobile(id_=UUID('c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7'), manufacturer='BMW', series_name='M4 Competition xDrive', type_=<AutomobileType.convertible: 'Convertible'>)

In [21]:
assert car.model_dump(by_alias=True) == expected_serialized_by_alias